In [1]:
import tensorflow as tf
import pandas as pd
import os

print("Script started.")


/Users/kalyanivaidya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Script started.


In [5]:
csv_path = "/Users/kalyanivaidya/AI Fashion Designer/data/DeepFashion2/paired_data.csv"
print("Loading CSV from:", csv_path)

pairs_df = pd.read_csv(csv_path)
print("CSV loaded. Number of pairs:", len(pairs_df))

sketch_paths = pairs_df['sketch'].tolist()
original_paths = pairs_df['original'].tolist()
print("Number of sketch paths:", len(sketch_paths))
print("Number of original paths:", len(original_paths))

Loading CSV from: /Users/kalyanivaidya/AI Fashion Designer/data/DeepFashion2/paired_data.csv
CSV loaded. Number of pairs: 191961
Number of sketch paths: 191961
Number of original paths: 191961


In [6]:
def load_and_preprocess_image(image_path, target_size=(256, 256), channels=3):
    print("Processing image:", image_path)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=channels)
    image = tf.image.resize(image, target_size)
    image = (image / 127.5) - 1.0
    return image

In [7]:
def load_pair(sketch_path, original_path):
    sketch = load_and_preprocess_image(sketch_path, target_size=(256, 256), channels=1)
    original = load_and_preprocess_image(original_path, target_size=(256, 256), channels=3)
    return sketch, original


In [8]:
dataset = tf.data.Dataset.from_tensor_slices((sketch_paths, original_paths))
print("Dataset created from file paths.")

Dataset created from file paths.


In [9]:
dataset = dataset.map(lambda sketch, orig: load_pair(sketch, orig),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
print("Mapping function applied to dataset.")


Processing image: Tensor("args_0:0", shape=(), dtype=string)
Processing image: Tensor("args_1:0", shape=(), dtype=string)
Mapping function applied to dataset.


In [10]:
batch_size = 4
dataset = dataset.batch(batch_size)
dataset = dataset.take(1)
dataset = dataset.prefetch(1)
print("Dataset batched, limited, and prefetched.")

Dataset batched, limited, and prefetched.


In [11]:
print("Iterating over dataset...")
for sketches, originals in dataset:
    print("Sketch batch shape:", sketches.shape)
    print("Original batch shape:", originals.shape)

print("Dataset processing completed.")

Iterating over dataset...
Sketch batch shape: (4, 256, 256, 1)
Original batch shape: (4, 256, 256, 3)
Dataset processing completed.


2025-05-07 16:02:14.334086: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
